In [1]:
import pandas as pd
from gensim.models.fasttext import FastText

In [2]:
df1 = pd.read_csv('finnum/train.csv')
x = df1.index
df2 = pd.read_csv('finnum/dev.csv')
df = pd.concat([df1,df2], ignore_index = True)
df.head(10)

,id,idx,tweet,category,subcategory,target_num
0,98221616,4976,$ARNA APD334 for Amyotrophic Lateral Sclerosis...,Product Number,Product Number,334
1,82321187,9839,"$OCLR Noob investor that i am, put a 7.38 stop...",Monetary,stop loss,7.38
2,103328840,1455,$ES_F $SPY Bias-2 bearish and the DLT-1 DRR ar...,Product Number,Product Number,1
3,104840294,1111,$TMUS its acquisition of Layer3 TV The purchas...,Product Number,Product Number,5
4,69935467,2373,$TWTR ^Buy $WSTL 68c up 14% 4 time avg vol. ...,Percentage,relative,14
5,69935467,2373,$TWTR ^Buy $WSTL 68c up 14% 4 time avg vol. ...,Quantity,Quantity,4
6,69935467,2373,$TWTR ^Buy $WSTL 68c up 14% 4 time avg vol. ...,Monetary,forecast,5
7,94249158,1372,$SEED L2 Capital deal is real savvy. It takes ...,Temporal,date,33
8,100979260,505,$BTE $BTE.CA $MEG.CA $CPG $CPG.CA $CJ.CA - 4th...,Temporal,date,4
9,100775772,1210,$WRN My fav $WRN pattern on my watchlist for 1...,Temporal,date,11


In [3]:
df['cat_num'] = df['category'].astype('category').cat.codes

In [4]:
model = FastText.load('fastText1')

In [5]:
df['lower'] = [x.lower() for x in df.tweet]

In [6]:
import re
def flagNum(x):
    text = x.lower
    outNum = str(x.target_num)
    text_out = re.sub(r'(?<=\D)'+outNum+'(?=\D)', ' <num> ', text)
    #text_out = text.replace('\D('+outNum+')\D', ' <num> ')
    return text_out

In [7]:
df['mod'] = df.apply(lambda x: flagNum(x), axis = 1)

In [8]:
import string
def textPuncandNum(text):
    table = str.maketrans('', '', string.punctuation)
    text = re.sub(r'[0-9]+', '', text)
    text = text.split()
    text = [word for word in text if len(word.translate(table))>2]
    return ' '.join(text)
stripped = [textPuncandNum(text) for text in df['mod']]

In [9]:
low = list(stripped)
low = ' '.join(low)
low = list(set(low.split()))
low.remove('<num>')
len(low)

14064

In [10]:
embed = model.wv[low]

In [11]:
label_dict = {k: v for v, k in enumerate(low)}
label_dict['<num>'] = len(label_dict) 

In [12]:
label_dict['<num>']

14064

In [13]:
bufferIndex = len(label_dict) 

In [14]:
import numpy as np
modifiedText = [[label_dict[word] for word in text.split()]for text in stripped]
maxLen = max(map(len, modifiedText))
for item in modifiedText:                # for each item in the list
    while len(item) < maxLen:            # while the item length is smaller than maxLen
        item.append(bufferIndex) 
numpyInp = np.asarray(modifiedText)

In [15]:
embed = np.vstack((embed, np.zeros(100)+20, np.zeros(100)+0))

In [16]:
train = numpyInp[x,:]
val = numpyInp[len(x):,:]

In [17]:
len(train)

6706

In [18]:
train.shape

(6706, 26)

In [19]:
import tensorflow as tf


In [20]:
tf.reset_default_graph()

In [21]:
trainInp = numpyInp[x,:]
valInp = numpyInp[len(x):,:]

labels = pd.get_dummies(df['cat_num'][x])
labels2 = pd.get_dummies(df['cat_num'][len(x):])

In [22]:
trainInp.shape

(6706, 26)

In [23]:
trainInp

array([[13708,  5775, 14064, ..., 14065, 14065, 14065],
       [11457, 12541,  2945, ..., 14065, 14065, 14065],
       [  534, 13329,  1200, ..., 14065, 14065, 14065],
       ...,
       [  579,   500,  8345, ..., 14065, 14065, 14065],
       [11401,  6428,  7171, ..., 14065, 14065, 14065],
       [14064, 11401,  6428, ..., 14065, 14065, 14065]])

In [24]:
batch_size = 32

train = (trainInp, labels)
val = (valInp, labels2)

# create training Dataset and batch it
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000) # if you want to shuffle your data
train_data = train_data.batch(batch_size)

# create validation Dataset and batch it
val_data = tf.data.Dataset.from_tensor_slices(val)
val_data = val_data.shuffle(10000) # if you want to shuffle your data
val_data = val_data.batch(batch_size)

# create one iterator and initialize it with different datasets
iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                           train_data.output_shapes)
txt, label = iterator.get_next()

train_init = iterator.make_initializer(train_data)
val_init = iterator.make_initializer(val_data)

In [25]:
embedding = tf.nn.embedding_lookup(embed, txt, partition_strategy='mod', name=None)
#embedded_chars_expanded = tf.expand_dims(embedding, -1)

In [26]:
embedding_out = tf.reshape(embedding, [26,100])

In [27]:
label.shape

TensorShape([Dimension(None), Dimension(7)])

In [28]:
n_steps = 26
n_inputs = 100
n_neurons = 150
n_outputs = 7

In [29]:
basic_cell = tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, embedding, dtype=tf.float64)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


In [30]:
learning_rate = 0.001
logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=label,logits=logits)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [33]:
logits = tf.cast(logits, tf.float32)
label = tf.cast(label, tf.int32)

In [34]:
correct = tf.nn.in_top_k(logits, label, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float64))

ValueError: Shape must be rank 1 but is rank 2 for 'in_top_k_1/InTopKV2' (op: 'InTopKV2') with input shapes: [?,7], [?,7], [].